In [3]:
library(tidyverse)
library(data.table)
library(magrittr)
library(harmony)
library(singlecellmethods)

In [2]:
normalizeData <- function(A, scaling_factor = 1e4, do_ftt = FALSE) {
	A@x <- A@x / rep.int(Matrix::colSums(A), diff(A@p))
    A@x <- scaling_factor * A@x
    if (do_ftt) {
        A@x <- sqrt(A@x) + sqrt(1 + A@x)
    } else {
        A@x <- log(1 + A@x)
    }
	return(A)
}

In [4]:
# X <- Matrix::t(harmony::ctrl.sparse)
X <- Matrix::cbind2(harmony::ctrl.sparse, harmony::stim.sparse)
X %<>% normalizeData()
X_ctrl <- normalizeData(harmony::ctrl.sparse)
X_stim <- normalizeData(harmony::stim.sparse)

In [5]:
system.time({
    .res <- findVariableGenes(X, c(rep("ctrl", ncol(harmony::ctrl.sparse)), rep("stim", ncol(harmony::stim.sparse))))
#     .res <- findVariableGenes(X_ctrl, rep(0, ncol(X_ctrl)))
})

.res %>% head

ERROR: Error in .Call("_singlecellmethods_exp_mean", PACKAGE = "singlecellmethods", : "_singlecellmethods_exp_mean" not available for .Call() for package "singlecellmethods"


Timing stopped at: 0.02 0 0.021


In [6]:
.x <- singlecellmethods:::exp_mean()

function (x, p, i, ncol, nrow, groups, group_sizes) 
{
    .Call("_singlecellmethods_exp_mean", PACKAGE = "singlecellmethods", 
        x, p, i, ncol, nrow, groups, group_sizes)
}
<bytecode: 0xcf63288>
<environment: namespace:singlecellmethods>

In [10]:
groups <- c(rep("ctrl", ncol(harmony::ctrl.sparse)), rep("stim", ncol(harmony::stim.sparse)))
groups <- factor(groups)
groups_int <- as.integer(factor(groups)) - 1
groups_table <- table(groups_int)
means_nonlog <- exp_mean(X@x, X@p, X@i, ncol(X), nrow(X), 
    groups_int, groups_table)

ERROR: Error in .Call("_singlecellmethods_exp_mean", PACKAGE = "singlecellmethods", : "_singlecellmethods_exp_mean" not available for .Call() for package "singlecellmethods"


In [ ]:
X